In [2]:
import numpy as np
import re

In [5]:
# estimate energy consumption using the simple energest output
# Example energest output:
# [INFO: Energest  ] --- Period summary #2 (60 seconds)
# [INFO: Energest  ] Total time  :    1966080
# [INFO: Energest  ] CPU         :      10374/   1966080 (5 permil)
# [INFO: Energest  ] LPM         :    1955706/   1966080 (994 permil)
# [INFO: Energest  ] Deep LPM    :          0/   1966080 (0 permil)
# [INFO: Energest  ] Radio Tx    :        106/   1966080 (0 permil)
# [INFO: Energest  ] Radio Rx    :     104802/   1966080 (53 permil)
# [INFO: Energest  ] Radio total :     104908/   1966080 (53 permil)

# Energest ouput
energest_output = " --- Period summary #2 (60 seconds)\nTotal time  :    1966080\nCPU         :      10374/   1966080 (5 permil)\nLPM         :    1955706/   1966080 (994 permil)\nDeep LPM    :          0/   1966080 (0 permil)\nRadio Tx    :        106/   1966080 (0 permil)\nRadio Rx    :     104802/   1966080 (53 permil)\nRadio total :     104908/   1966080 (53 permil)"

# with regex, we can extract the numbers from the output
# total time
# CPU time
# LPM time
# Radio Tx time
# Radio Rx time
# Radio total time

# regex to extract the numbers
regex = re.compile(r"Energest\s+]\s\D+\s+:\s+(\d+)")

# extract the values
match = regex.findall(energest_output)
times = [int(x) for x in match]

print(times)

total_time, cpu_time, lpm_time, deep_lpm_time, radio_tx_time, radio_rx_time, radio_total_time = times

# print the values
print("Total time: %s" % total_time)
print("CPU time: %s" % cpu_time)
print("LPM time: %s" % lpm_time)
print("Deep LPM: %s" % deep_lpm_time)
print("Radio Tx time: %s" % radio_tx_time)
print("Radio Rx time: %s" % radio_rx_time)
print("Radio total time: %s" % radio_total_time)

# current draw in different states in mA
# CPU_CURRENT = 10
# LPM_CURRENT = 0.023
# DEEP_LPM_CURRENT = 0
# RADIO_TX_CURRENT = 18.4
# RADIO_RX_CURRENT = 17.4

CURRENT = {
    "CPU": 10,
    "LPM": 0.023,
    "DEEP_LPM": 0,
    "RADIO_TX": 18.4,
    "RADIO_RX": 17.4
}

# 3 volt battery
VOLTAGE = 3

# We have the following values available
# []_time - the number of ticks spent in each mode
# TICKS_PER_SECOND - the number of ticks per second
# total_time - the total time in ticks
# []_CURRENT - the current draw in each mode
# voltage - the provided voltage to the component

# Equations
# state_avg_current_mA = ([]_time * current_mA) / total_time
# state_charge_mC = ([]_time * current_mA) / TICKS_PER_SECOND
# state_charge_mAh = state_charge_mC / 3600
# state_power_mW = avg_current_mA * voltage
# state_energy_mJ = state_charge_mC * voltage

TICKS_PER_SECOND = 32768

# calculate the average current in each state
states_values = { f"{state}":
    {
        f"avg_current_mA": (state_time * state_current) / total_time,
        f"charge_mC": (state_time * state_current) / TICKS_PER_SECOND,
        f"charge_mAh": (state_time * state_current) / TICKS_PER_SECOND / 3600,
        f"power_mW": (state_time * state_current) / total_time * VOLTAGE,
        f"energy_mJ": (state_time * state_current) / TICKS_PER_SECOND * VOLTAGE
    } for state_time, state_current, state in zip(times[:-1], CURRENT.values(), CURRENT.keys())
}

# from values in each state, we can calculate the total charge consumption and energy consumption 
total_charge_mC = 0
total_charge_mAh = 0
total_energy_mJ = 0
for state in CURRENT.keys():
    total_charge_mC += states_values[f"{state}"][f"charge_mC"] 
    total_charge_mAh += states_values[f"{state}"][f"charge_mAh"]
    total_energy_mJ += states_values[f"{state}"][f"energy_mJ"]

# print the values
print(f"{total_charge_mC:.2f} mC ({total_charge_mAh:.2f} mAh) charge consumption, {total_energy_mJ:.2f} mJ energy consumption in {total_time / TICKS_PER_SECOND:.2f} seconds")




[]


ValueError: not enough values to unpack (expected 7, got 0)